In [1]:
using BenchmarkTools, Plots, JLD2
using ITensors, LinearAlgebra, KrylovKit
#I have based my code on this reference: https://journals.aps.org/prb/abstract/10.1103/PhysRevB.91.165112

In [45]:
bdag=[0 0
    1 0]

id=[1 0
    0 1]

br=kron(bdag,id)
bc=kron(id,bdag)
brc=kron(bdag,bdag)
bid=kron(id,id)
#Note that the Hilbert space for hardcore boson is (no boson,one boson).
vecvacuum=kron([1,0],[1,0]) #no bosons r,c
vecr=kron([0,1],[1,0])
vecc=kron([1,0],[0,1])
vecrc=kron([0,1],[0,1])

function readout(matrix,vecleft,vecright,dim)
basisvec=Matrix(I,dim,dim)
    operator=zeros(ComplexF64,dim,dim)
    for i in 1:dim
        for j in 1:dim
            operator[i,j]=dot(kron(vecleft,basisvec[i,:]),matrix*kron(vecright,basisvec[j,:]))
        end
    end
    return operator
end


function computeWII(Hmpo::MPO,deltatime,sites)
length=size(Hmpo)[1]
sqrtdeltatime=sqrt(deltatime)
WII_array=Array{Any}(undef,length)
wmat=array(Hmpo[1],commonind(Hmpo[1],Hmpo[2]),prime(sites[1]),sites[1])
WII=zeros(ComplexF64,size(wmat)[1]-1,dim(sites[1]),dim(sites[1]))
for i=1:(size(wmat)[1]-2)
    Cmat=wmat[i+1,:,:]
    Dmat=wmat[1,:,:]
    hext=sqrtdeltatime*kron(bc,Cmat)+deltatime*kron(bid,Dmat)
    wfull=exp(hext)#Then let's read out the relevant part with at most one boson for each species, but how?
    WII[1,:,:]=readout(wfull,vecvacuum,vecvacuum,dim(sites[1])) #D
    WII[i+1,:,:]=readout(wfull,vecc,vecvacuum,dim(sites[1])) #C
end
WII_array[1]=WII

for sitei=2:length-1
    wmat=array(Hmpo[sitei],commonind(Hmpo[sitei-1],Hmpo[sitei]),commonind(Hmpo[sitei],Hmpo[sitei+1]),prime(sites[sitei]),sites[sitei])
    WII=zeros(ComplexF64,size(wmat)[1]-1,size(wmat)[2]-1,dim(sites[sitei]),dim(sites[sitei]))
    for i=1:(size(wmat)[1]-2)
        for j=1:(size(wmat)[2]-2)
            Amat=wmat[i+1,j+1,:,:]
            Bmat=wmat[i+1,1,:,:]
            Cmat=wmat[size(wmat)[1],j+1,:,:]
            Dmat=wmat[size(wmat)[1],1,:,:]  
            hext=kron(brc,Amat)+sqrtdeltatime*kron(br,Bmat)+sqrtdeltatime*kron(bc,Cmat)+deltatime*kron(bid,Dmat)
            wfull=exp(hext)#Then let's read out the relevant part with at most one boson for each species, but how?
            WII[1,1,:,:]=readout(wfull,vecvacuum,vecvacuum,dim(sites[sitei])) #D
            WII[1,j+1,:,:]=readout(wfull,vecc,vecvacuum,dim(sites[sitei]))  #C
            WII[1+i,1,:,:]=readout(wfull,vecr,vecvacuum,dim(sites[sitei])) #B
            WII[1+i,1+j,:,:]=readout(wfull,vecrc,vecvacuum,dim(sites[sitei])) #A
        end
    end
    WII_array[sitei]=WII
end

wmat=array(Hmpo[length],commonind(Hmpo[length-1],Hmpo[length]),prime(sites[length]),sites[length])
WII=zeros(ComplexF64,size(wmat)[1]-1,dim(sites[length]),dim(sites[length]))
for i=1:(size(wmat)[1]-2)
    Dmat=wmat[size(wmat)[1],:,:]
    Bmat=wmat[i+1,:,:]   
    hext=sqrtdeltatime*kron(br,Bmat)+deltatime*kron(bid,Dmat)
    wfull=exp(hext)#Then let's read out the relevant part with at most one boson for each species, but how?
    WII[1,:,:]=readout(wfull,vecvacuum,vecvacuum,dim(sites[length])) #D
    WII[1+i,:,:]=readout(wfull,vecr,vecvacuum,dim(sites[length])) #B
end
WII_array[length]=WII
    
link=Array{Index}(undef,length-1)
for i=1:length-1
    link[i]=Index(size(WII_array[i+1])[1],"Link,l=$i")
end

WIItensor=Array{ITensor}(undef,length)
WIItensor[1]=ITensor(WII_array[1],link[1],prime(sites[1]), sites[1])
for i=2:length-1
    WIItensor[i]=ITensor(WII_array[i],link[i-1],link[i],prime(sites[i]), sites[i])
end

WIItensor[length]=ITensor(WII_array[length],link[length-1],prime(sites[length]), sites[length])

Udt=deepcopy(Hmpo)
for i=1:length
    Udt[i]=WIItensor[i]
end
    return Udt
end


computeWII (generic function with 2 methods)

In [3]:
uc = 2; ## sites/unit cell
L = 3*uc; ## sites in chain; need 3 unit cells to get MPO from middle unit
sites = siteinds("S=1/2", L); ## creates array of L site indices with dimensions of S=1/2

## 1D Kitaev
Jx = 2.; Jy = 1.;

os = OpSum(); ## create empty operator sum to add Hamiltonian terms
for j = 1:2:L-1;
    ## add in Ising bond terms; factor of 4 to offset 1/2 in Sz definition
    os += (4*Jx, "Sx", j, "Sx", j+1);
    if j+2<=L
        os += (4*Jy, "Sy", j+1, "Sy", j+2);
    end
end

Ham = MPO(os, sites); ## MPO with 6 indices (2 per site + 2)

In [36]:
psi0=randomMPS(sites,1);
energy,psi=dmrg(Ham, psi0; nsweeps=40, maxdim=1200, cutoff=1e-10);

After sweep 1 energy=-6.2497669180448545  maxlinkdim=4 maxerr=2.82E-16 time=0.049
After sweep 2 energy=-6.249770839242331  maxlinkdim=4 maxerr=3.45E-11 time=0.011
After sweep 3 energy=-6.249770839262308  maxlinkdim=4 maxerr=3.31E-11 time=0.007
After sweep 4 energy=-6.249770839324219  maxlinkdim=4 maxerr=3.03E-11 time=0.009
After sweep 5 energy=-6.249770839359988  maxlinkdim=4 maxerr=2.03E-11 time=0.100
After sweep 6 energy=-6.2497708393918625  maxlinkdim=4 maxerr=1.93E-11 time=0.007
After sweep 7 energy=-6.249770839465076  maxlinkdim=4 maxerr=1.49E-11 time=0.005
After sweep 8 energy=-6.249770839472555  maxlinkdim=4 maxerr=7.34E-12 time=0.006
After sweep 9 energy=-6.249770839505781  maxlinkdim=4 maxerr=5.99E-12 time=0.005
After sweep 10 energy=-6.249770839527902  maxlinkdim=3 maxerr=8.14E-11 time=0.004
After sweep 11 energy=-6.249770839528116  maxlinkdim=3 maxerr=1.34E-13 time=0.005
After sweep 12 energy=-6.2497708395283595  maxlinkdim=3 maxerr=1.16E-13 time=0.006
After sweep 13 energy=

In [46]:
#This is the third-order WII approximation
thirdalpha=0.10566243
thirdbeta=0.39433756
delta1=thirdalpha-im*thirdbeta
delta2=thirdbeta+im*thirdalpha
delta3=thirdbeta-im*thirdalpha
delta4=thirdalpha+im*thirdbeta

time_evolve_mpoI=computeWII(Ham,-0.01*im*delta1,sites)
time_evolve_mpoII=computeWII(Ham,-0.01*im*delta2,sites)
time_evolve_mpoIII=computeWII(Ham,-0.01*im*delta3,sites)
time_evolve_mpoIV=computeWII(Ham,-0.01*im*delta4,sites)


#Or you can try the following parameters for a second-order WII approximation   

#    seconddelta1=(1+im)/2
#    seconddelta2=(1-im)/2
#    WII1=computeWII(H,-deltat*im*seconddelta1,sites)
#    WII2=computeWII(H,-deltat*im*seconddelta2,sites)

MPO
[1] ((dim=2|id=20|"Link,l=1"), (dim=2|id=913|"S=1/2,Site,n=1")', (dim=2|id=913|"S=1/2,Site,n=1"))
[2] ((dim=2|id=20|"Link,l=1"), (dim=2|id=597|"Link,l=2"), (dim=2|id=119|"S=1/2,Site,n=2")', (dim=2|id=119|"S=1/2,Site,n=2"))
[3] ((dim=2|id=597|"Link,l=2"), (dim=2|id=330|"Link,l=3"), (dim=2|id=698|"S=1/2,Site,n=3")', (dim=2|id=698|"S=1/2,Site,n=3"))
[4] ((dim=2|id=330|"Link,l=3"), (dim=2|id=961|"Link,l=4"), (dim=2|id=102|"S=1/2,Site,n=4")', (dim=2|id=102|"S=1/2,Site,n=4"))
[5] ((dim=2|id=961|"Link,l=4"), (dim=2|id=463|"Link,l=5"), (dim=2|id=100|"S=1/2,Site,n=5")', (dim=2|id=100|"S=1/2,Site,n=5"))
[6] ((dim=2|id=463|"Link,l=5"), (dim=2|id=512|"S=1/2,Site,n=6")', (dim=2|id=512|"S=1/2,Site,n=6"))


In [47]:
psievolve=deepcopy(psi)
for i=1:100
    print(i)
    psievolve=apply(time_evolve_mpoIV, psievolve; cutoff=1e-12,maxdim=400)
    psievolve=apply(time_evolve_mpoIII, psievolve; cutoff=1e-12,maxdim=400)
    psievolve=apply(time_evolve_mpoII, psievolve; cutoff=1e-12,maxdim=400)
    psievolve=apply(time_evolve_mpoI, psievolve; cutoff=1e-12,maxdim=400)
end

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100

In [48]:
abs(inner(psievolve,psi)) 
#Check the ground state fidelity after time evolution. The error should be (dt)^4*1/dt=dt^3, which is indeed 10^-6 in this case

0.9999988611118107